## Intro:

First of all, we would like to thank very much the organizers of this hackathon Zindi and Yassir and all participants, we were amazed to see that there was a very good number of data scientists in Algeria.

This notebook is a bit long to run, we removed all unnecessary parts and EDA since it's not required. 

For the catboost model it takes around 20-30 mins to finish and XGBoost around 40 mins to run. During the hackathon, each team member was running a different model on his machine to gain time.

## Installing Catboost


For this notebook we need to install catboost first in order to train later 3 models: XGBoost, Catboost and Random Forest 

In [ ]:
!pip install catboost

     |████████████████████████████████| 64.8MB 60kB/s 


# Feature Engineering & Cross Validation

In [ ]:
import numpy as np #scientific computing 
import pandas as pd  #data analysis and manipulation

import matplotlib.pyplot as plt 
import matplotlib as mpl
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go 
from plotly.subplots import make_subplots 

from sklearn import decomposition, cluster, preprocessing, model_selection, metrics

from catboost import CatBoostRegressor

import os
import gc 
import sys 
import time 
from datetime import datetime 

import warnings 
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics 
from sklearn import cluster
from sklearn import model_selection 
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn import pipeline 
from sklearn.compose import ColumnTransformer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In order for results to be reproducible, we will choose a number for random state and the path of train.csv and test.csv file along with other provided files from zindi.

In [ ]:
# Make sure to specify the path to the data containing Test.csv,Train.csv,SampleSubmission.csv,Weather.csv
np.random.RandomState(42)
path = '/content/drive/My Drive/umoja/'

## Helper Functions

We will define several helper functions useful for feature engineering:

- Haversine_array which compute a list of haversine distance for the full data (train + test) through this well known formula:

$$haversine(\theta)=sin^2\Big(\frac{\theta}{2}\Big)$$

$$ haversine\Big(\frac{d}{r}\Big)=haversine(\Phi_2-\Phi_1)+ cos(\Phi_1)cos(\Phi_2)haversine(\lambda_2-\lambda_1)$$

- Manhatten distance which is given by: 

$${\displaystyle d(A,B)=|X_{B}-X_{A}|+|Y_{B}-Y_{A}|}$$

- A function that transforms lat_origin, lon_origin to point 1 (x,y) and lat_dest, lon_dest to point 2 (x,y).

## Feature Engineering:

The function reset_data, creates several useful features (around 70 features), some interesting one are:

- Clustering origin and destination to k clusters (after several trials, k = 300 gives best result) 
- Mean, min and max ETA by cluster of origin and destination and by hour group. Same thing for average speed 

- PCA of lat and longitude to get better splits since we are using decision trees based models.

- Time related features like hour, day, month, weekend..etc and cycle preserving features like hour_cos and month_sin so that 1 AM is almost same as 11 PM (23h vs 1h).

- Number of trips per cluster.

- Direction of each trip calculated through lat_origin, lon_origin, lat_dest and lon_dest.


And few more features.


In [ ]:
# Distance functions
def haversine_array(lat1, lng1, lat2, lng2):
    #lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    lat1 = np.radians(lat1)
    lng1 = np.radians(lng1)
    lat2 = np.radians(lat2)
    lng2 = np.radians(lng2)
    
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def dummy_manhattan_distance(lat1, lng1, lat2, lng2):
    a = haversine_array(lat1, lng1, lat1, lng2)
    b = haversine_array(lat1, lng1, lat2, lng1)
    return a + b

def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    #lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    lat1 = np.radians(lat1)
    lng1 = np.radians(lng1)
    lat2 = np.radians(lat2)
    lng2 = np.radians(lng2)
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

# A function that generates the datafile with all the feature extractions
def reset_data(n_clusters):
    train = pd.read_csv(path + 'Train.csv')
    test = pd.read_csv(path + 'Test.csv')
    weather = pd.read_csv(path + 'Weather.csv')

    data = pd.concat([train, test])

    # Timestamp
    data.Timestamp = pd.to_datetime(data.Timestamp).dt.tz_convert(None)
    
    data['date'] = data['Timestamp'].dt.date
    data.date = pd.to_datetime(data.date)
    '''
    data['dayofyear'] = data.date.dt.dayofyear

    weather.date = pd.to_datetime(weather.date)
    weather['dayofyear'] = weather.date.dt.dayofyear

    data = data.merge(weather, on = 'dayofyear', how = 'left')
    data.drop(['dayofyear', 'date_x', 'date_y'], axis = 1, inplace=True)
    
    # Weather features
    data['temperature_bin'] = pd.qcut(data.mean_2m_air_temperature, q=4)

    label = preprocessing.LabelEncoder()
    data['temperature_bin'] = label.fit_transform(data['temperature_bin'])

    data['precipitation_bin'] = data.total_precipitation.apply(precipitation_bin)
    '''
    # Time features
    data['weekday'] = data['Timestamp'].dt.weekday
    data['weekday_name'] = data['Timestamp'].dt.day_name()
    data['dayofmonth'] = data['Timestamp'].dt.day
    data['hour'] = data['Timestamp'].dt.hour
    data['minute'] = data['Timestamp'].dt.minute
    data['weekhour'] = data['weekday'] * 24 + data['hour']

    # Cyclical patterns features
    data['dayofmonth_sin'] = np.sin(data['dayofmonth']*(2.*np.pi/31)) 
    data['dayofmonth_cos'] = np.cos(data['dayofmonth']*(2.*np.pi/31)) 
    data['weekday_sin'] = np.sin(data['weekday']*(2.*np.pi/7)) 
    data['weekday_cos'] = np.cos(data['weekday']*(2.*np.pi/7)) 
    data['hour_sin'] = np.sin(data['hour']*(2.*np.pi/23)) 
    data['hour_cos'] = np.cos(data['hour']*(2.*np.pi/23))  
    data['minute_sin'] = np.sin(data['minute']*(2.*np.pi/60))  
    data['minute_cos'] = np.cos(data['minute']*(2.*np.pi/60)) 
    
    data["weekend"] = data["weekday"].apply(lambda x: 1 if x == 5 else 1 if x == 4 else 0)

    data['hour_bin'] = 0
    data.loc[(data['hour']>=19)&(data['hour']<=23),'hour_bin']= 1
        
    # Lat / Lon
    coords = np.vstack((data[['Origin_lat', 'Origin_lon']].values, data[['Destination_lat', 'Destination_lon']].values))

    # PCA
    pca = decomposition.PCA().fit(coords)

    data['Origin_pca0'] = pca.transform(data[['Origin_lat', 'Origin_lon']])[:, 0]
    data['Origin_pca1'] = pca.transform(data[['Origin_lat', 'Origin_lon']])[:, 1]
    data['Destination_pca0'] = pca.transform(data[['Destination_lat', 'Destination_lon']])[:, 0]
    data['Destination_pca1'] = pca.transform(data[['Destination_lat', 'Destination_lon']])[:, 1]

    # Distances
    data['distance_haversine'] = haversine_array(data['Origin_lat'].values, data['Origin_lon'].values, data['Destination_lat'].values, data['Destination_lon'].values)
    data['distance_dummy_manhattan'] = dummy_manhattan_distance(data['Origin_lat'].values, data['Origin_lon'].values, data['Destination_lat'].values, data['Destination_lon'].values)
    data['direction'] = bearing_array(data['Origin_lat'].values, data['Origin_lon'].values, data['Destination_lat'].values, data['Destination_lon'].values)
    data['pca_manhattan'] = np.abs(data['Destination_pca1'] - data['Origin_pca1']) + np.abs(data['Destination_pca0'] - data['Origin_pca0'])

    data['center_lat'] = (data['Origin_lat'].values + data['Destination_lat'].values) / 2
    data['center_lon'] = (data['Origin_lon'].values + data['Destination_lon'].values) / 2

    data['Origin_lat_bin'] = np.round(data['Origin_lat'], 2)
    data['Origin_lon_bin'] = np.round(data['Origin_lon'], 2)

    data['Destination_lat_bin'] = np.round(data['Destination_lat'], 2)
    data['Destination_lon_bin'] = np.round(data['Destination_lon'], 2)

    data['center_lat_bin'] = np.round(data['center_lat'], 2)
    data['center_lon_bin'] = np.round(data['center_lon'], 2)

    # Speeds
    data['avg_speed_h'] = 1000 * data['distance_haversine'] / data['ETA']
    data['avg_speed_m'] = 1000 * data['distance_dummy_manhattan'] / data['ETA']
    data['avg_speed'] = data['Trip_distance'] / data['ETA']

    speeds = ['avg_speed','avg_speed_h', 'avg_speed_m']
    gby_cols = ['Origin_lat_bin', 'Origin_lon_bin']

    # speeds per Origin bin
    speeds = ['avg_speed','avg_speed_h', 'avg_speed_m']
    gby_cols = ['Origin_lat_bin', 'Origin_lon_bin']

    coord_speed = data.groupby(gby_cols).mean()[speeds].rename(columns = {x : x + '_bin' for x in speeds})
    for speed in coord_speed.columns:
        coord_speed[speed].fillna(coord_speed[speed].median(), inplace = True)

    coord_speed = coord_speed.reset_index()
    data = data.merge(coord_speed, on = gby_cols, how ='left')

    # Kmeans Clustering
    sample_ind = np.random.permutation(len(coords))
    kmeans = cluster.MiniBatchKMeans(n_clusters=n_clusters, batch_size=10000).fit(coords[sample_ind])

    data['Origin_cluster'] = kmeans.predict(data[['Origin_lat', 'Origin_lon']])
    data['Destination_cluster'] = kmeans.predict(data[['Destination_lat', 'Destination_lon']])

    cluster_lat = np.array(kmeans.cluster_centers_)[:, 0]
    cluster_lon = np.array(kmeans.cluster_centers_)[:, 1]

    Origin_cluster_center = pd.DataFrame(data = np.transpose([cluster_lat, cluster_lon]), index = data.Origin_cluster.unique(), columns=['Origin_cluster_lat', 'Origin_cluster_lon']).reset_index().rename(columns = {'index':'Origin_cluster'})
    Destination_cluster_center = pd.DataFrame(data = np.transpose([cluster_lat, cluster_lon]), index = data.Origin_cluster.unique(), columns=['Destination_cluster_lat', 'Destination_cluster_lon']).reset_index().rename(columns = {'index':'Destination_cluster'})

    data = data.merge(Origin_cluster_center, on = 'Origin_cluster', how='left')
    data = data.merge(Destination_cluster_center, on = 'Destination_cluster', how='left')
    
    # Clustering related Distances
    data['distance_haversine_cluster'] = haversine_array(data['Origin_cluster_lat'].values, data['Origin_cluster_lon'].values, data['Destination_cluster_lat'].values, data['Destination_cluster_lon'].values)
    data['distance_dummy_manhattan_cluster'] = dummy_manhattan_distance(data['Origin_cluster_lat'].values, data['Origin_cluster_lon'].values, data['Destination_cluster_lat'].values, data['Destination_cluster_lon'].values)
    data['direction_cluster'] = bearing_array(data['Origin_cluster_lat'].values, data['Origin_cluster_lon'].values, data['Destination_cluster_lat'].values, data['Destination_cluster_lon'].values)
    
    # Some Groupby features
    gby_cols = ['hour', 'weekhour', 'weekday', 'Origin_cluster', 'Destination_cluster']
    for gby_col in gby_cols:
        gby = data.groupby(gby_col).mean()[['avg_speed_h', 'avg_speed_m', 'avg_speed', 'ETA']]
        gby.columns = ['%s_gby_%s' % (col, gby_col) for col in gby.columns]
        data = pd.merge(data, gby, how='left', left_on=gby_col, right_index=True)

    # Min grby cluster
    for gby_col in gby_cols:
        gby = data.groupby(gby_col).min()[['avg_speed_h', 'avg_speed_m', 'avg_speed', 'ETA']]
        gby.columns = ['%s_min_gby_%s' % (col, gby_col) for col in gby.columns]
        data = pd.merge(data, gby, how='left', left_on=gby_col, right_index=True)
    # max
    for gby_col in gby_cols:
        gby = data.groupby(gby_col).max()[['avg_speed_h', 'avg_speed_m', 'avg_speed', 'ETA']]
        gby.columns = ['%s_max_gby_%s' % (col, gby_col) for col in gby.columns]
        data = pd.merge(data, gby, how='left', left_on=gby_col, right_index=True)

    # Counting features
    group_freq = '60min'
    data['Timestamp_group'] = data['Timestamp'].dt.round(group_freq)

    # Count trips over 60min
    df_counts = data.set_index('Timestamp')[['ID']].sort_index()
    df_counts['count_60min'] = df_counts.isnull().rolling(group_freq).count()['ID']
    data = data.merge(df_counts, on='ID', how='left')

    # Count how many trips are going from each cluster over time
    Origin_counts = data[['ID', 'Timestamp', 'Origin_cluster', 'Destination_cluster']] \
        .set_index('Timestamp') \
        .groupby([pd.Grouper(freq=group_freq), 'Origin_cluster']) \
        .agg({'ID': 'count'}) \
        .reset_index().set_index('Timestamp') \
        .groupby('Origin_cluster').rolling('240min').mean() \
        .drop('Origin_cluster', axis=1) \
        .reset_index().set_index('Timestamp').shift(freq='-120min').reset_index() \
        .rename(columns={'Timestamp': 'Timestamp_group', 'ID': 'Origin_cluster_count'})

    data['Origin_cluster_count'] = data[['Timestamp_group', 'Origin_cluster']].merge(Origin_counts, on=['Timestamp_group', 'Origin_cluster'], how='left')['Origin_cluster_count'].fillna(0)

    # Count how many trips are arriving in each cluster over time
    Destination_counts = data[['ID', 'Timestamp', 'Origin_cluster', 'Destination_cluster']] \
        .set_index('Timestamp') \
        .groupby([pd.Grouper(freq=group_freq), 'Destination_cluster']) \
        .agg({'ID': 'count'}) \
        .reset_index().set_index('Timestamp') \
        .groupby('Destination_cluster').rolling('240min').mean() \
        .drop('Destination_cluster', axis=1) \
        .reset_index().set_index('Timestamp').shift(freq='-120min').reset_index() \
        .rename(columns={'Timestamp': 'Timestamp_group', 'ID': 'Destination_cluster_count'})

    data['Destination_cluster_count'] = data[['Timestamp_group', 'Destination_cluster']].merge(Destination_counts, on=['Timestamp_group', 'Destination_cluster'], how='left')['Destination_cluster_count'].fillna(0)
    return data



## Cross Validation & Other Helper functions


The function test_features take a list of features and generate X, y and X_test (from test.csv) with these features.

Submission function generate a submission file.

**Cross Validation Strategy:**

An important thing to notice is that test is the future of train, so we cannot shuffle data in classic KFold cross validation, so we are using TimeSeriesSplit to respect the time dependence and chronological order of the data.






In [ ]:
def test_features(features):
    train = data[:83924].set_index("Timestamp")
    test = data[83924:].set_index("Timestamp")

    X = train[features]
    y = train["ETA"]
    X_test = test[features]
    return X, y, X_test

def submission(model): 
    model.fit(X, y)
    y_preds = model.predict(X_test)
    sub = pd.read_csv(path + 'SampleSubmission.csv')
    sub["ETA"] = y_preds
    return sub

# Make cross val through a time based approach, help to avoid overfitting on LB
def cross_val(m, X, y, k=4):
    # Timestamp should be the index
    tscv = model_selection.TimeSeriesSplit(k)
    X = X.sort_index().reset_index().drop('Timestamp',1)
    y = y.sort_index().reset_index().drop('Timestamp',1)
    print("sorted index")
    cv_errors = []
    for train_index, test_index in tscv.split(X):
        print(X.index)
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]
        m.fit(X_train,y_train)
        y_preds = m.predict(X_test)
        fold_err = metrics.mean_squared_error(y_test, y_preds, squared=False)
        print("Fold 1, done, error:", fold_err)
        cv_errors.append(fold_err)
    print("Mean CV error: ", sum(cv_errors)/len(cv_errors))

#Catboost Model

We will start by training a catboost model using a subset of created features. This subset of features was chosen through several trials using cross validation.

Note: reset_data(300) will generate all hand created features using k=300 for number of clusters

In [ ]:
data = reset_data(300)

# We used a catboot model on all features to generate the important features
features_simp = ['Trip_distance', 'ETA_gby_Origin_cluster',
       'ETA_gby_Destination_cluster', 'avg_speed_gby_Destination_cluster',
       'distance_dummy_manhattan', 'avg_speed_bin',
       'avg_speed_gby_Origin_cluster', 'distance_haversine',
       'avg_speed_h_gby_Destination_cluster', 'pca_manhattan',
       'ETA_gby_weekhour', 'Destination_pca1',
       'avg_speed_m_gby_Destination_cluster', 'Origin_lon',
       'avg_speed_gby_weekhour', 'Destination_pca0', 'minute',
       'Origin_pca1', 'Origin_pca0', 'Destination_lon', 'Origin_lat',
       'direction', 'Destination_lat', 'center_lat', 'center_lon_bin',
       'count_60min', 'center_lon', 'avg_speed_m_bin',
       'Destination_lon_bin', 'dayofmonth_cos', 'hour_sin',
       'avg_speed_h_bin', 'ETA_gby_hour', 'Destination_cluster',
       'Destination_cluster_lat', 'hour_cos', 'ETA_min_gby_hour', 'ETA_min_gby_Origin_cluster', 
       'ETA_min_gby_Destination_cluster',
       'ETA_max_gby_hour','ETA_max_gby_Origin_cluster','ETA_max_gby_Destination_cluster'
       ]

X, y, X_test = test_features(features_simp)

Using Random Search over several parameters we get the following parameters. we then train a CatBoostRegressor using these params

In [ ]:
# Fined tuned a catboost model using randomsearch
# 150.68 on LB
param = {'per_float_feature_quantization': '4', 
           'depth' : 10,
           'iterations': 8000,
           'l2_leaf_reg': 1,
           'learning_rate': 0.02,
           'random_state': 42,
           'od_type':'IncToDec'}

model_cat_tune = CatBoostRegressor(**param)
model_cat_tune.fit(X, y, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
3001:	learn: 85.0130239	total: 8m 37s	remaining: 14m 22s
3002:	learn: 85.0002283	total: 8m 38s	remaining: 14m 22s
3003:	learn: 84.9918681	total: 8m 38s	remaining: 14m 21s
3004:	learn: 84.9865504	total: 8m 38s	remaining: 14m 21s
3005:	learn: 84.9799930	total: 8m 38s	remaining: 14m 21s
3006:	learn: 84.9718080	total: 8m 38s	remaining: 14m 21s
3007:	learn: 84.9575698	total: 8m 38s	remaining: 14m 21s
3008:	learn: 84.9518319	total: 8m 39s	remaining: 14m 21s
3009:	learn: 84.9422025	total: 8m 39s	remaining: 14m 21s
3010:	learn: 84.9331337	total: 8m 39s	remaining: 14m 21s
3011:	learn: 84.9263963	total: 8m 39s	remaining: 14m 20s
3012:	learn: 84.9203605	total: 8m 40s	remaining: 14m 20s
3013:	learn: 84.9069071	total: 8m 40s	remaining: 14m 20s
3014:	learn: 84.8914007	total: 8m 40s	remaining: 14m 20s
3015:	learn: 84.8808316	total: 8m 40s	remaining: 14m 20s
3016:	learn: 84.8680405	total: 8m 41s	remaining: 14m 20s
3017:	lear

Let's save the y predictions of the fine tuned cat boost model we just trained to blend it later with other models

In [ ]:
y_preds_cat_tuned = model_cat_tune.predict(X_test)
'''
sub_cat_tuned = pd.read_csv(path + 'SampleSubmission.csv')
sub_cat_tuned["ETA"] = y_preds_cat_tuned
'''
#sub_cat_tuned.to_csv('Brahim_cat_tuned_7.csv', index = False)

#RandomForestRegressor Model

For Random Forest regressor, we will use a different set of features which was selected through cross validation with 4 folds.

In [ ]:
features_2= ['Trip_distance', 'ETA_gby_Origin_cluster',
       'ETA_gby_Destination_cluster', 'pca_manhattan', 'avg_speed_bin',
       'avg_speed_gby_Destination_cluster', 'Destination_pca1',
       'avg_speed_h_gby_Destination_cluster',
       'avg_speed_gby_Origin_cluster', 'Destination_lon',
       'ETA_gby_weekhour', 'avg_speed_m_gby_Destination_cluster',
       'Origin_pca1', 'distance_haversine', 'Origin_lon',
       'Destination_pca0', 'Origin_pca0', 'Destination_lat', 'minute',
       'distance_dummy_manhattan', 'Origin_lat', 'avg_speed_gby_weekhour',
       'center_lon', 'direction', 'distance_haversine_cluster',
       'distance_dummy_manhattan_cluster', 'center_lat',
       'avg_speed_h_bin', 'avg_speed_m_bin', 'Destination_lat_bin',
       'Destination_cluster', 'count_60min', 'Destination_cluster_lat',
       'hour_sin', 'avg_speed_h_gby_hour', 'Destination_cluster_lon',
       'ETA_gby_hour', 'center_lon_bin', 'avg_speed_m_gby_weekhour',
       'dayofmonth_cos']             

Same as catboost, we generate all the features using 300 cluster then we select the features using `features_2` list.

In [ ]:
data = reset_data(300)
X, y, X_test = test_features(features_2)


Random forest is extremly slow so we will just use a simple train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_testing, y_train, y_testing=train_test_split(X,y,test_size=0.2,random_state=42)

Training a default Random Forest Regressor.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rand_forest_regressor = RandomForestRegressor(random_state=42)
rand_forest_regressor.fit(X_train, y_train)

y_rand_forest_predict = rand_forest_regressor.predict(X_test)


#XGBoost Model

We will do exactly the same for XGBoost model, we got the parameters through a random search and setting n_estimators to 8000 since we have a large number of rows.

In [ ]:
from xgboost import XGBRegressor
XGB_model=XGBRegressor(learning_rate=0.05,max_depth=7,n_estimators=8000,random_state=42)
XGB_model.fit(X_train,y_train)
y_XGB_predict=XGB_model.predict(X_test)


[21:34:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


#Blending The results

Now, for the final step, we will blend all results, using a weighted average. we got the weights through several submissions on LB.

In [ ]:
y=0.3*y_XGB_predict+0.55*y_preds_cat_tuned+0.15*y_rand_forest_predict

#Submission

Generating a submission file.

In [ ]:
sub = pd.read_csv(path + 'SampleSubmission.csv')
sub["ETA"] = y
sub.to_csv('Submission.csv', index = False)